In [6]:
import numpy as np
import copy

import hyperopt as hpo
from hyperopt.fmin import generate_trials_to_calculate

In [7]:
from hyperopt import hp
space = {
    'width': hp.uniform('width', 0, 20),
    'height': hp.uniform('height', -100, 100),
    'activation': hp.choice("activation", ["relu", "tanh"])
}

# current_best_params = [
#     {
#         "width": 1,
#         "height": 2,
#         "activation": 0  # Activation will be relu
#     },
#     {
#         "width": 4,
#         "height": 2,
#         "activation": 1  # Activation will be tanh
#     }
# ]

config = {
    "my_exp": {
        "run": "exp",
        "num_samples": 10,
        "config": {
            "iterations": 100,
        },
        "stop": {
            "timesteps_total": 100
        },
    }
}

# Hyper Optimizer

In [188]:
class HyperOptSearch():

    def __init__(self,
                 space,
                 max_concurrent=10,
                 reward_attr="episode_reward_mean",
                 points_to_evaluate=None,
                 **kwargs):
        assert hpo is not None, "HyperOpt must be installed!"
        assert type(max_concurrent) is int and max_concurrent > 0
        self._max_concurrent = max_concurrent
        self._reward_attr = reward_attr
        self.algo = hpo.tpe.suggest
        self.domain = hpo.Domain(lambda spc: spc, space)
        if points_to_evaluate is None:
            self._hpopt_trials = hpo.Trials()
            self._points_to_evaluate = 0
        else:
            assert type(points_to_evaluate) == list
            self._hpopt_trials = generate_trials_to_calculate(
                points_to_evaluate)
            self._hpopt_trials.refresh()
            self._points_to_evaluate = len(points_to_evaluate)
        self._live_trial_mapping = {}
        self.rstate = np.random.RandomState()

        super(HyperOptSearch, self).__init__(**kwargs)

    def _suggest(self, trial_id):
        if self._num_live_trials() >= self._max_concurrent:
            return None

        if self._points_to_evaluate > 0:
            new_trial = self._hpopt_trials.trials[self._points_to_evaluate - 1]
            self._points_to_evaluate -= 1
        else:
            new_ids = self._hpopt_trials.new_trial_ids(1)
            self._hpopt_trials.refresh()

            # Get new suggestion from Hyperopt
            new_trials = self.algo(new_ids, self.domain, self._hpopt_trials,
                                   self.rstate.randint(2**31 - 1))
            self._hpopt_trials.insert_trial_docs(new_trials)
            self._hpopt_trials.refresh()
            new_trial = new_trials[0]
        self._live_trial_mapping[trial_id] = (new_trial["tid"], new_trial)

        # Taken from HyperOpt.base.evaluate
        config = hpo.base.spec_from_misc(new_trial["misc"])
        ctrl = hpo.base.Ctrl(self._hpopt_trials, current_trial=new_trial)
        memo = self.domain.memo_from_config(config)
        hpo.utils.use_obj_for_literal_in_memo(self.domain.expr, ctrl,
                                              hpo.base.Ctrl, memo)

        suggested_config = hpo.pyll.rec_eval(
            self.domain.expr,
            memo=memo,
            print_node_on_error=self.domain.rec_eval_print_node_on_error)
        return copy.deepcopy(suggested_config)

    def on_trial_result(self, trial_id, result):
        ho_trial = self._get_hyperopt_trial(trial_id)
        if ho_trial is None:
            return
        now = hpo.utils.coarse_utcnow()
        ho_trial['book_time'] = now
        ho_trial['refresh_time'] = now

    def on_trial_complete(self,
                          trial_id,
                          result=None,
                          error=False,
                          early_terminated=False):
        """Passes the result to HyperOpt unless early terminated or errored.
        The result is internally negated when interacting with HyperOpt
        so that HyperOpt can "maximize" this value, as it minimizes on default.
        """
        ho_trial = self._get_hyperopt_trial(trial_id)
        if ho_trial is None:
            return
        ho_trial['refresh_time'] = hpo.utils.coarse_utcnow()
        if error:
            ho_trial['state'] = hpo.base.JOB_STATE_ERROR
            ho_trial['misc']['error'] = (str(TuneError), "Tune Error")
        elif early_terminated:
            ho_trial['state'] = hpo.base.JOB_STATE_ERROR
            ho_trial['misc']['error'] = (str(TuneError), "Tune Removed")
        else:
            ho_trial['state'] = hpo.base.JOB_STATE_DONE
            hp_result = self._to_hyperopt_result(result)
            ho_trial['result'] = hp_result
        self._hpopt_trials.refresh()
        del self._live_trial_mapping[trial_id]

    def _to_hyperopt_result(self, result):
        return {"loss": -result[self._reward_attr], "status": "ok"}

    def _get_hyperopt_trial(self, trial_id):
        if trial_id not in self._live_trial_mapping:
            return
        hyperopt_tid = self._live_trial_mapping[trial_id][0]
        return [
            t for t in self._hpopt_trials.trials if t["tid"] == hyperopt_tid
        ][0]

    def _num_live_trials(self):
        return len(self._live_trial_mapping)

# Evaluation Function

In [200]:
def easy_objective(config):
    neg_mean_loss = -(config["height"] - 14)**2 + abs(config["width"] - 3)
    result = {"neg_mean_loss": neg_mean_loss}
    return result

# Sequential example

### Initializer Search algo

In [201]:
# algo = HyperOptSearch(
#     space,
#     max_concurrent=4,
#     reward_attr="neg_mean_loss",
#     points_to_evaluate=current_best_params)

algo = HyperOptSearch(
    space,
    max_concurrent=1,
    reward_attr="neg_mean_loss")

trial_id = 'testing'

In [202]:
print(algo._num_live_trials()); print()
print(algo._live_trial_mapping); print()
print(algo._hpopt_trials.__dict__); print()

0

{}

{'_ids': set(), '_dynamic_trials': [], '_exp_key': None, 'attachments': {}, '_trials': []}



### Fetch suggestion for trial by provided trial_id

In [203]:
suggestion = algo._suggest(trial_id)
print(suggestion)

{'activation': 'tanh', 'height': -1.3731605607652853, 'width': 0.6153860935314515}


In [204]:
print(algo._num_live_trials()); print()
print(algo._live_trial_mapping); print()
print(algo._get_hyperopt_trial(trial_id)); print()

1

{'testing': (0, {'state': 0, 'tid': 0, 'spec': None, 'result': {'status': 'new'}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'activation': [0], 'height': [0], 'width': [0]}, 'vals': {'activation': [1], 'height': [-1.3731605607652853], 'width': [0.6153860935314515]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': None, 'refresh_time': None})}

{'state': 0, 'tid': 0, 'spec': None, 'result': {'status': 'new'}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'activation': [0], 'height': [0], 'width': [0]}, 'vals': {'activation': [1], 'height': [-1.3731605607652853], 'width': [0.6153860935314515]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': None, 'refresh_time': None}



### Evaluate suggested hyperparameters

In [205]:
result = easy_objective(suggestion)
print(result)

{'neg_mean_loss': -233.94945172060065}


### Report result


In [206]:
algo._hpopt_trials._dynamic_trials

[{'state': 0,
  'tid': 0,
  'spec': None,
  'result': {'status': 'new'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'activation': [0], 'height': [0], 'width': [0]},
   'vals': {'activation': [1],
    'height': [-1.3731605607652853],
    'width': [0.6153860935314515]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': None,
  'refresh_time': None}]

In [207]:
# book_time & refresh_time are set upon returning the result
algo.on_trial_result(trial_id, result)
algo._hpopt_trials._dynamic_trials

[{'state': 0,
  'tid': 0,
  'spec': None,
  'result': {'status': 'new'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'activation': [0], 'height': [0], 'width': [0]},
   'vals': {'activation': [1],
    'height': [-1.3731605607652853],
    'width': [0.6153860935314515]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2019, 2, 21, 22, 47, 25, 929000),
  'refresh_time': datetime.datetime(2019, 2, 21, 22, 47, 25, 929000)}]

In [208]:
algo._hpopt_trials.__dict__

{'_ids': {0},
 '_dynamic_trials': [{'state': 0,
   'tid': 0,
   'spec': None,
   'result': {'status': 'new'},
   'misc': {'tid': 0,
    'cmd': ('domain_attachment', 'FMinIter_Domain'),
    'workdir': None,
    'idxs': {'activation': [0], 'height': [0], 'width': [0]},
    'vals': {'activation': [1],
     'height': [-1.3731605607652853],
     'width': [0.6153860935314515]}},
   'exp_key': None,
   'owner': None,
   'version': 0,
   'book_time': datetime.datetime(2019, 2, 21, 22, 47, 25, 929000),
   'refresh_time': datetime.datetime(2019, 2, 21, 22, 47, 25, 929000)}],
 '_exp_key': None,
 'attachments': {},
 '_trials': [{'state': 0,
   'tid': 0,
   'spec': None,
   'result': {'status': 'new'},
   'misc': {'tid': 0,
    'cmd': ('domain_attachment', 'FMinIter_Domain'),
    'workdir': None,
    'idxs': {'activation': [0], 'height': [0], 'width': [0]},
    'vals': {'activation': [1],
     'height': [-1.3731605607652853],
     'width': [0.6153860935314515]}},
   'exp_key': None,
   'owner': Non

In [209]:
# Resets refresh_time & updates result
print(algo.on_trial_complete(trial_id, result)); print()
algo._hpopt_trials.__dict__

None



{'_ids': {0},
 '_dynamic_trials': [{'state': 2,
   'tid': 0,
   'spec': None,
   'result': {'loss': 233.94945172060065, 'status': 'ok'},
   'misc': {'tid': 0,
    'cmd': ('domain_attachment', 'FMinIter_Domain'),
    'workdir': None,
    'idxs': {'activation': [0], 'height': [0], 'width': [0]},
    'vals': {'activation': [1],
     'height': [-1.3731605607652853],
     'width': [0.6153860935314515]}},
   'exp_key': None,
   'owner': None,
   'version': 0,
   'book_time': datetime.datetime(2019, 2, 21, 22, 47, 25, 929000),
   'refresh_time': datetime.datetime(2019, 2, 21, 22, 47, 26, 426000)}],
 '_exp_key': None,
 'attachments': {},
 '_trials': [{'state': 2,
   'tid': 0,
   'spec': None,
   'result': {'loss': 233.94945172060065, 'status': 'ok'},
   'misc': {'tid': 0,
    'cmd': ('domain_attachment', 'FMinIter_Domain'),
    'workdir': None,
    'idxs': {'activation': [0], 'height': [0], 'width': [0]},
    'vals': {'activation': [1],
     'height': [-1.3731605607652853],
     'width': [0.61

### One more iteration

In [210]:
suggestion = algo._suggest(trial_id)
result = easy_objective(suggestion)
algo.on_trial_result(trial_id, result)
algo.on_trial_complete(trial_id, result)
algo._hpopt_trials._dynamic_trials

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': 233.94945172060065, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'activation': [0], 'height': [0], 'width': [0]},
   'vals': {'activation': [1],
    'height': [-1.3731605607652853],
    'width': [0.6153860935314515]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2019, 2, 21, 22, 47, 25, 929000),
  'refresh_time': datetime.datetime(2019, 2, 21, 22, 47, 26, 426000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': 2301.8326375575343, 'status': 'ok'},
  'misc': {'tid': 1,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'activation': [1], 'height': [1], 'width': [1]},
   'vals': {'activation': [0],
    'height': [-34.1279148420155],
    'width': [17.4635494827617]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2019, 2, 21

In [212]:
print(algo._hpopt_trials._dynamic_trials[0]['result']['loss'])
print(algo._hpopt_trials._dynamic_trials[1]['result']['loss'])

233.94945172060065
2301.8326375575343


### Run search 100 times more

In [215]:
for _ in range(100):
    suggestion = algo._suggest(trial_id)
    result = easy_objective(suggestion)
    algo.on_trial_result(trial_id, result)
    algo.on_trial_complete(trial_id, result)
    print(algo._hpopt_trials._dynamic_trials[-1]['result']['loss'])

9.06693588824082
132.59070293413217
423.1460380750672
90.3692272883436
-15.335153910146333
546.7765739645824
258.6725747577753
85.69453611262799
462.7212141975487
58.4740758857037
-14.911356873749023
11.650968889147158
200.00080527025452
185.87631757766428
56.734119222977554
654.8831094320271
-14.30217986786031
626.9450385922133
321.621714925711
4.783081071491576
83.56297923289557
921.3633576986615
840.9873962553777
328.971974647292
132.02523847383966
-10.361497093333021
7.7519036349118835
285.05034821529745
6.193462274942185
140.02748790911718
477.6702174758122
1289.3997019818676
-15.631806982566982
483.06921407831726
71.91043717618464
1146.117978141471
28.776390284462245
248.27240693093415
201.4370620674202
-5.956475757574834
119.6319100996355
58.868613482588245
644.9716701861198
781.4664165364215
1497.0333741137642
49.68702236408042
430.23062816286404
-16.723076043487115
17.0828140716995
-7.603057123494974
11993.470833643712
871.7663979074869
393.37697398913366
266.8096694159708
106